In [ ]:
correlation_columns = ['pH', 'EC', 'OC' ,
                    'P', 'K', 'S',
                    'Zn' ,'Fe', 
                    'Mn', 'Cu']
correlation_data_frame = pd.DataFrame(soil_database_df.iloc[ :, [6,7,8,10,11,12,13,15,16 ,17]  ]  )
correlation_data_frame.columns = correlation_columns
correlation_data_frame = correlation_data_frame.replace('--',np.NaN)
correlation_data_frame = correlation_data_frame.astype(float,errors='raise')
soil_cr = correlation_data_frame.corr(method='pearson')
display(soil_cr)
correlation_data_frame.to_pickle('soil_health_data')

In [ ]:
#Plots for correlation in soil nutrients
sb.set(style="white")

# Generate a mask for the upper triangle
mask = np.zeros_like(soil_cr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sb.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sb.heatmap(soil_cr, mask=mask, cmap=cmap, vmax=.3, center=0, annot=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
display(correlation_data_frame.describe())

In [ ]:
#Scatter matrix depicting correlation of soil nutrients
pd.plotting.scatter_matrix(
    correlation_data_frame,
    figsize  = [15, 15],
#     marker   = ".",
#     s        = 0.2,
#     diagonal = "kde"
)

In [ ]:
#soil_database_df.info()

In [ ]:
#soil_database_df.describe()

In [ ]:
#import tensorflow as tf
from sklearn import metrics
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [ ]:
correlation_data_frame = correlation_data_frame.reindex(np.random.permutation(correlation_data_frame.index))
# correlation_data_frame.describe()
X = correlation_data_frame[['pH', 'EC', 'OC', 
                     'S', 'Zn', 'Fe', 
                    'Mn','Cu']].copy()
Y_Phosphorus = correlation_data_frame[['P']].copy()
X.columns = ['pH', 'EC', 'OC', 
                     'S', 'Zn', 'Fe', 
                    'Mn','Cu']
Y_Phosphorus.columns = ['P']

In [ ]:
b = X.loc[ :, 'pH' : 'Cu' ] == np.NaN
b.sum()


In [ ]:
X = X.replace(np.NaN,0)
Y_Phosphorus = Y_Phosphorus.replace(np.NaN,0)

In [ ]:
# display(X, Y_Phosphorus)
seed = 123
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, Y_Phosphorus, test_size=test_size, random_state=seed)

In [ ]:
seed = 123
valid_size = 0.2
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=valid_size, random_state=seed)

In [ ]:
print(X_train.shape , X_valid.shape , X_test.shape)

In [ ]:
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)
valid_matrix = xgb.DMatrix(data=X_valid)
test_matrix  = xgb.DMatrix(data=X_test,label=y_test)


In [ ]:
def try_xgb_params(data_matrix,test_matrix  , num_boost_round=10 ,  early_stopping_rounds=50, **params):
    model = xgb.train(params,  data_matrix   )
    y_pred = model.predict( test_matrix)
    return y_pred , model

In [ ]:
def get_accuracy( model , y_pred , y , X   ):
    rmse = np.sqrt(metrics.mean_squared_error(y , y_pred))
    return rmse 

In [ ]:
params1 = {"objective":"reg:linear",'colsample_bytree': 0.2,'learning_rate': 0.6,
                'max_depth': 4, 'alpha': 20 }

In [ ]:
y_pred , model_try = try_xgb_params(data_dmatrix ,valid_matrix ,   params1)

In [ ]:
rmse  =   get_accuracy(model_try ,y_pred , y_valid , X_valid)

In [ ]:
print(rmse )

In [ ]:
xgb.plot_importance(model)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()6.24520772357577

In [ ]:
xgb.plot_tree(model)
plt.rcParams['figure.figsize'] = [200, 200]
# fig = plt.gcf() 
# fig.set_size_inches(150, 100)
plt.show()

In [ ]:
#model.get_score(X_valid , y_valid)


list_of_params = []
params1 = {"objective":"reg:linear",'colsample_bytree': 0.2,'learning_rate': 0.30,
                'max_depth': 4, 'alpha': 10 }
params2 = {"objective":"reg:linear",'colsample_bytree': 0.4,'learning_rate': 0.5,
                'max_depth': 5, 'alpha': 10 }
params3 = {"objective":"reg:linear",'colsample_bytree': 0.8,'learning_rate': 0.7,
                'max_depth': 6, 'alpha': 10 }

list_of_params.append(params1)
list_of_params.append(params2)
list_of_params.append(params3)

for param in list_of_params:
    del model
    y_pred , model = try_xgb_params(data_dmatrix ,valid_matrix ,   param)
    rmse  =   get_accuracy(model ,y_pred , y_valid , X_valid)
    print(rmse)


In [ ]:
# model = XGBRegressor(objective ='reg:linear', colsample_bytree = 1, learning_rate = 0.1,
#                 max_depth = 4, alpha = 10, n_estimators = 8)
# model.fit(X_train, y_train)
# print(model)
# #predict
# y_pred = model.predict(X_test)
# # print(y_test, y_pred)
# # evaluate predictions
# print(Y_Phosphorus.describe())
# rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
# print("RMSE: %f" % (rmse))
# print(model.score(X_train, y_train))
# # predictions = [round(value) for value in y_pred]
# # evaluate predictions
# # print("Accuracy: %.2f%%" % (accuracy * 100.0))
# xgb.plot_importance(model)
# plt.rcParams['figure.figsize'] = [5, 5]
# plt.show()

In [ ]:
# # xgb.plot_tree(columns = ['pH', 'EC', 'Organic Carbon (OC)', 
# #                      'Available Sulphur (S)', 'Available Zinc (Zn)', 'Available Iron (Fe)', 
# #                     'Available Manganese (Mn)','Available Copper (Cu)'], model = model.booster, trees = None)
# data_dmatrix = xgb.DMatrix(data=X,label=Y_Phosphorus)
# params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
#                 'max_depth': 5, 'alpha': 10}
# # xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10)
# xgb.plot_tree(model)
# plt.rcParams['figure.figsize'] = [100, 100]
# plt.show()

<h4>  convert into validation set

In [ ]:
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33, random_state=seed)



# print(X_train.shape)
# print(X_valid.shape)